# Check for confounding variables

This notebook uses chi-squared tests to look for clinical variables that are associated with having a chromosome event or not.

- Get clinical tables
- Get event tables
- Binarize clinical columns as needed
- For each binary column in the clinical table, make a contingency table of that column and the event table
- Run chi squared test and save results

In [1]:
import pandas as pd
import numpy as np
import os
import cptac
import altair as alt
import scipy.stats
import statsmodels.stats.multitest

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

In [3]:
dss = {
    "brca": cptac.Brca,
#     "ccrcc": cptac.Ccrcc,
    "colon": cptac.Colon,
#     "endometrial": cptac.Endometrial,
#     "gbm": cptac.Gbm,
    "hnscc": cptac.Hnscc,
    "lscc": cptac.Lscc,
    "luad": cptac.Luad,
    "ovarian": cptac.Ovarian
}

In [4]:
def load_tables(cancer_type):
    
    # Load the dataset
    ds = dss[cancer_type]()
    
    # Get the clinical table
    clin = ds.get_clinical()
    
    # Get the event table
    event = pd.\
    read_csv(f"{cancer_type}_has_event.tsv", sep="\t", index_col=0).\
    rename(columns={"gain_event": "8q_gain", "loss_event": "8p_loss"})
    
    joined = clin.join(event, how="inner")
    
    return joined

In [5]:
def test_cnv_association(cancer_type, df, test_cols, cnv_col):
    
    pvals = []

    for col in test_cols:
        
        # Create contingency table
        contingency_table = pd.crosstab(df[cnv_col], df[col])
        
        # Run test
        chi2, p, dof, exp_freq = scipy.stats.chi2_contingency(contingency_table)
        
        # Check assumptions: No group has expected value < 1, and no more than
        # 20% of groups have expected frequency < 5.
        exp_freq = pd.DataFrame(exp_freq)
        
        if (exp_freq < 1).any().any():
            pvals.append("Not all expected frequencies were > 1.")
        elif (exp_freq < 5).sum().sum() > 0.2 * exp_freq.shape[0] * exp_freq.shape[1]:
            pvals.append("More than 20% of groups had expected frequency < 5.")
        else:
            pvals.append(p)
        
    pvals = pd.DataFrame({
        "cancer_type": cancer_type,
        "cnv_event": cnv_col,
        "variable": test_cols,
        "pval": pvals
    })
    
    return pvals

In [6]:
all_results = pd.DataFrame()

## BRCA

In [7]:
brca = load_tables("brca")

### Simplify the age column
For the age column, we will create groups of 15 years, and combine all > 75 years.

In [8]:
brca = brca.assign(Age=brca["Age.in.Month"] // 12)
brca = brca.assign(Age_group=(brca["Age"] // 15) * 15)
brca = brca.assign(Age_group=brca["Age_group"].where(cond=(brca["Age_group"] < 75) | (pd.isnull(brca["Age"])), other=75))

In [9]:
brca["Age_group"].value_counts(dropna=False).sort_index()

30.0    12
45.0    36
60.0    38
75.0    19
NaN     17
Name: Age_group, dtype: int64

### Simplify the stage column

We will also simplify the "Stage" column.

In [10]:
brca["Stage"].value_counts(dropna=False).sort_index()

Stage IA       4
Stage IIA     50
Stage IIB     20
Stage III      4
Stage IIIA    22
Stage IIIB     3
Stage IIIC     4
NaN           15
Name: Stage, dtype: int64

Because there are only 4 Stage I samples, we will group them with Stage II.

In [11]:
def simplify_stage_brca(row):
    if pd.isna(row):
        return row
    elif row.startswith("Stage III"):
        return "III"
    elif row.startswith("Stage II"):
        return "I or II"
    elif row.startswith("Stage I"):
        return "I or II"
    else:
        return row
    
brca = brca.assign(Stage=brca["Stage"].apply(simplify_stage_brca))

In [12]:
brca["Stage"].value_counts(dropna=False).sort_index()

I or II    74
III        33
NaN        15
Name: Stage, dtype: int64

### Race column

There aren't enough people in the hispanic.or.latino group to satisfy the requirements of the chi-squared test, so we have to drop the category.

In [13]:
brca["Race"].value_counts(dropna=False)

white                        78
asian                        19
black.or.african.american    14
NaN                           7
hispanic.or.latino            4
Name: Race, dtype: int64

In [14]:
brca["Race"] = brca["Race"].replace(
    to_replace="hispanic.or.latino",
    value=np.nan
)

In [15]:
brca["Race"].value_counts(dropna=False)

white                        78
asian                        19
black.or.african.american    14
NaN                          11
Name: Race, dtype: int64

### Run chi-squared tests
Now we will run chi-squared tests to look for association between each variable and CNV events.

In [16]:
brca_cols = [
    "Age_group",
    "Race",
    "Stage",
    "PAM50",
    "NMF.v2.1",
]
# Don't use gender because all female

In [17]:
all_results = all_results.append(test_cnv_association(
    cancer_type="brca",
    df=brca,
    test_cols=brca_cols,
    cnv_col="8p_loss"
))

In [18]:
all_results = all_results.append(test_cnv_association(
    cancer_type="brca",
    df=brca,
    test_cols=brca_cols,
    cnv_col="8q_gain"
))

## Colon

In [19]:
colon = load_tables("colon")

### Simplify the Age column

In [20]:
colon = colon.assign(Age_years=colon["Age"] // 12)
colon = colon.assign(Age_group=(colon["Age_years"] // 15) * 15)

In [21]:
colon["Age_group"].value_counts(dropna=False).sort_index()

30.0     4
45.0    31
60.0    48
75.0    19
90.0     1
NaN      2
Name: Age_group, dtype: int64

In [22]:
colon = colon.assign(
    Age_group=colon["Age_group"].where(cond=(colon["Age_group"] < 75) | (pd.isnull(colon["Age"])), other=75)
)
colon = colon.assign(
    Age_group=colon["Age_group"].where(cond=(colon["Age_group"] > 45) | (pd.isnull(colon["Age"])), other=30)
)

In [23]:
colon["Age_group"].value_counts(dropna=False).sort_index()

30.0    35
60.0    48
75.0    20
NaN      2
Name: Age_group, dtype: int64

### Simplify the Stage column

In [24]:
colon["Stage"].value_counts(dropna=False).sort_index()

Stage I      12
Stage II     42
Stage III    44
Stage IV      7
Name: Stage, dtype: int64

In [25]:
colon = colon.assign(Stage=colon["Stage"].where(colon["Stage"] != "Stage IV", "Stage III or IV"))
colon = colon.assign(
    Stage=colon["Stage"].where(colon["Stage"] != "Stage III", "Stage III or IV")
)

In [26]:
colon["Stage"].value_counts(dropna=False).sort_index()

Stage I            12
Stage II           42
Stage III or IV    51
Name: Stage, dtype: int64

### Run chi-squared tests

In [27]:
colon_cols = [
    "Age_group",
    "Gender",
    "Stage",
    "Mucinous"
]

In [28]:
all_results = all_results.append(test_cnv_association(
    cancer_type="colon",
    df=colon,
    test_cols=colon_cols,
    cnv_col="8p_loss"
))

In [29]:
all_results = all_results.append(test_cnv_association(
    cancer_type="colon",
    df=colon,
    test_cols=colon_cols,
    cnv_col="8q_gain"
))

## HNSCC

In [30]:
hnscc = load_tables("hnscc")

cptac warning: The HNSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (/home/caleb/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py, line 4)


### Group ages

In [31]:
hnscc = hnscc.assign(Age_group=(hnscc["age"] // 10) * 10)

In [32]:
hnscc["Age_group"].value_counts(dropna=False).sort_index()

20.0     1
40.0     5
50.0    37
60.0    48
70.0    14
80.0     3
NaN      1
Name: Age_group, dtype: int64

In [33]:
hnscc = hnscc.assign(
    Age_group=hnscc["Age_group"].where(cond=(hnscc["Age_group"] < 70) | (pd.isnull(hnscc["age"])), other=70)
)
hnscc = hnscc.assign(
    Age_group=hnscc["Age_group"].where(cond=(hnscc["Age_group"] > 50) | (pd.isnull(hnscc["age"])), other=50)
)

In [34]:
hnscc["Age_group"].value_counts(dropna=False).sort_index()

50.0    43
60.0    48
70.0    17
NaN      1
Name: Age_group, dtype: int64

### Simplify alcohol consumption column

We are going to combine the past drinker group with the current but less group.

Also replace the history not available group with NaN.

In [35]:
hnscc["alcohol_consum"].value_counts(dropna=False)

Alcohol consumption equal to or less than 2 drinks per day for men and 1 drink or less per day for women    44
Alcohol consumption history not available                                                                   23
Lifelong non-drinker                                                                                        21
Alcohol consumption more than 2 drinks per day for men and more than 1 drink per day for women              11
NaN                                                                                                          7
Consumed alcohol in the past, but currently a non-drinker                                                    3
Name: alcohol_consum, dtype: int64

In [36]:
hnscc["alcohol_consum"] = hnscc["alcohol_consum"].replace(
    to_replace="Consumed alcohol in the past, but currently a non-drinker",
    value="Alcohol consumption equal to or less than 2 drinks per day for men and 1 drink or less per day for women"
).replace(
    to_replace="Alcohol consumption history not available",
    value=np.nan
)

In [37]:
hnscc["alcohol_consum"].value_counts(dropna=False)

Alcohol consumption equal to or less than 2 drinks per day for men and 1 drink or less per day for women    47
NaN                                                                                                         30
Lifelong non-drinker                                                                                        21
Alcohol consumption more than 2 drinks per day for men and more than 1 drink per day for women              11
Name: alcohol_consum, dtype: int64

### Simplify smoking history column

Combine all the "current reformed" groups and set the "history not available" group to NaN.

In [38]:
hnscc["smoking_history"].value_counts(dropna=False)

Current smoker: Includes daily and non-daily smokers                38
Smoking history not available                                       21
Lifelong non-smoker: Less than 100 cigarettes smoked in lifetime    21
Current reformed smoker within past 15 years                        14
Current reformed smoker, more than 15 years                         10
Current reformed smoker, years unknown                               4
NaN                                                                  1
Name: smoking_history, dtype: int64

In [39]:
hnscc["smoking_history"] = hnscc["smoking_history"].replace(
    to_replace="Smoking history not available",
    value=np.nan
).replace(
    to_replace=[
        "Current reformed smoker, years unknown", 
        "Current reformed smoker within past 15 years",
        "Current reformed smoker, more than 15 years"
    ],
    value="Current reformed smoker"
)

In [40]:
hnscc["smoking_history"].value_counts(dropna=False)

Current smoker: Includes daily and non-daily smokers                38
Current reformed smoker                                             28
NaN                                                                 22
Lifelong non-smoker: Less than 100 cigarettes smoked in lifetime    21
Name: smoking_history, dtype: int64

### Simplify tumor site column

Combine the two pharynx categories, and put lip with Oral cavity.

In [41]:
hnscc["tumor_site_curated"].value_counts(dropna=False)

Oral cavity    49
Larynx         47
Oropharynx      6
Lip             4
Hypopharynx     2
NaN             1
Name: tumor_site_curated, dtype: int64

In [42]:
hnscc["tumor_site_curated"] = hnscc["tumor_site_curated"].replace(
    to_replace=["Oropharynx", "Hypopharynx"],
    value="Pharynx"
).replace(
    to_replace="Lip",
    value="Oral cavity"
)

In [43]:
hnscc["tumor_site_curated"].value_counts(dropna=False)

Oral cavity    53
Larynx         47
Pharynx         8
NaN             1
Name: tumor_site_curated, dtype: int64

### Simplify stage column

Combine stage I and stage II groups

In [44]:
hnscc["patho_staging_curated"].value_counts(dropna=False)

Stage IV     45
Stage III    32
Stage II     24
Stage I       7
NaN           1
Name: patho_staging_curated, dtype: int64

In [45]:
hnscc["patho_staging_curated"] = hnscc["patho_staging_curated"].replace(
    to_replace=["Stage I", "Stage II"],
    value="Stage I/II"
)

In [46]:
hnscc["patho_staging_curated"].value_counts(dropna=False)

Stage IV      45
Stage III     32
Stage I/II    31
NaN            1
Name: patho_staging_curated, dtype: int64

### Run chi-squared tests

In [47]:
hnscc_cols = [
    "Age_group",
    "alcohol_consum",
#     "gender", # There are only 14 women and 94 men. Chi square assumption not met: More than 20% of groups had expected frequency < 5.
    "histologic_grade",
#     "histologic_type", # 97 out of 104 are all "Squamous cell carcinoma, conventional"
    "patho_staging_curated",
    "smoking_history",
    "tumor_site_curated"
]

In [48]:
all_results = all_results.append(test_cnv_association(
    cancer_type="hnscc",
    df=hnscc,
    test_cols=hnscc_cols,
    cnv_col="8p_loss"
))

In [49]:
all_results = all_results.append(test_cnv_association(
    cancer_type="hnscc",
    df=hnscc,
    test_cols=hnscc_cols,
    cnv_col="8q_gain"
))

## LSCC

In [50]:
lscc = load_tables("lscc")

cptac warning: The LSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (/home/caleb/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py, line 4)


In [51]:
lscc["Age"] = lscc["Age"].astype(int)

### Group ages

In [52]:
lscc = lscc.assign(Age_group=(lscc["Age"] // 10) * 10)

In [53]:
lscc["Age_group"].value_counts(dropna=False).sort_index()

40     5
50    18
60    46
70    37
80     2
Name: Age_group, dtype: int64

In [54]:
lscc = lscc.assign(
    Age_group=lscc["Age_group"].where(cond=(lscc["Age_group"] < 70) | (pd.isnull(lscc["Age"])), other=70)
)
lscc = lscc.assign(
    Age_group=lscc["Age_group"].where(cond=(lscc["Age_group"] > 50) | (pd.isnull(lscc["Age"])), other=50)
)

In [55]:
lscc["Age_group"].value_counts(dropna=False).sort_index()

50    23
60    46
70    39
Name: Age_group, dtype: int64

### Simplify the Stage column

In [56]:
lscc["Stage"].value_counts(dropna=False).sort_index()

I        3
IA      12
IB      22
II       2
IIA     22
IIB     20
III      2
IIIA    17
IIIB     2
IV       1
NaN      5
Name: Stage, dtype: int64

In [57]:
def simplify_stage_lscc(row):
    if pd.isna(row):
        return row
    elif row.startswith("IV"):
        return "III or IV"
    elif row.startswith("III"):
        return "III or IV"
    elif row.startswith("II"):
        return "II"
    elif row.startswith("I"):
        return "I"
    else:
        return row
    
lscc = lscc.assign(Stage=lscc["Stage"].apply(simplify_stage_lscc))

In [58]:
lscc["Stage"].value_counts(dropna=False).sort_index()

I            37
II           44
III or IV    22
NaN           5
Name: Stage, dtype: int64

### Country.of.Origin column

Combine by region--North America, Asian, or Eurasia. We use this column instead of the Ethnicity column because the Ethnicity column only had enough info test two groups--white and Asian.

In [59]:
lscc["Country.of.Origin"].value_counts(dropna=False).sort_index()

bulgaria    25
china       17
other        2
poland      14
russia       2
ukraine     10
usa         32
vietnam      6
Name: Country.of.Origin, dtype: int64

In [60]:
lscc["Country.of.Origin"] = lscc["Country.of.Origin"].replace(
    to_replace="usa",
    value="North America"
).replace(
    to_replace=["bulgaria", "poland", "russia", "ukraine"],
    value="Eurasia"
).replace(
    to_replace=["china", "vietnam"],
    value="Asia"
).replace(
    to_replace="other",
    value=np.nan
)

In [61]:
lscc["Country.of.Origin"].value_counts(dropna=False).sort_index()

Asia             23
Eurasia          51
North America    32
NaN               2
Name: Country.of.Origin, dtype: int64

### Pack.Years.Smoked column

In [62]:
lscc["Pack.Years.Smoked"] = lscc["Pack.Years.Smoked"].astype(float)

lscc = lscc.assign(**{"Pack.Years.Smoked": (lscc["Pack.Years.Smoked"] // 20) * 20})

lscc = lscc.assign(**{
    "Pack.Years.Smoked": lscc["Pack.Years.Smoked"].where(
        cond=(lscc["Pack.Years.Smoked"] < 60) | (pd.isnull(lscc["Pack.Years.Smoked"])), other=60
    )
})
lscc = lscc.assign(**{
    "Pack.Years.Smoked": lscc["Pack.Years.Smoked"].where(
        cond=(lscc["Pack.Years.Smoked"] > 20) | (pd.isnull(lscc["Pack.Years.Smoked"])), other=20
    )
})

In [63]:
lscc["Pack.Years.Smoked"].value_counts(dropna=False).sort_index()

20.0    23
40.0    20
60.0    14
NaN     51
Name: Pack.Years.Smoked, dtype: int64

### Run chi squared tests

In [64]:
lscc_cols = [
    "Age_group",
    "Gender",
    "Stage",
    "Country.of.Origin",
    "Pack.Years.Smoked"
]

In [65]:
all_results = all_results.append(test_cnv_association(
    cancer_type="lscc",
    df=lscc,
    test_cols=lscc_cols,
    cnv_col="8p_loss"
))

In [66]:
all_results = all_results.append(test_cnv_association(
    cancer_type="lscc",
    df=lscc,
    test_cols=lscc_cols,
    cnv_col="8q_gain"
))

## LUAD

In [67]:
luad = load_tables("luad")

### Group ages

In [68]:
luad = luad.assign(Age_group=(luad["Age"] // 10) * 10)

In [69]:
luad["Age_group"].value_counts(dropna=False).sort_index()

30.0     2
40.0    10
50.0    32
60.0    39
70.0    23
80.0     3
Name: Age_group, dtype: int64

In [70]:
luad = luad.assign(
    Age_group=luad["Age_group"].where(cond=(luad["Age_group"] < 70) | (pd.isnull(luad["Age"])), other=70)
)
luad = luad.assign(
    Age_group=luad["Age_group"].where(cond=(luad["Age_group"] > 40) | (pd.isnull(luad["Age"])), other=40)
)

In [71]:
luad["Age_group"].value_counts(dropna=False).sort_index()

40.0    12
50.0    32
60.0    39
70.0    26
Name: Age_group, dtype: int64

### Simplify BMI column

Based on the ranges from the CDC: https://www.cdc.gov/obesity/adult/defining.html

In [72]:
luad = luad.assign(BMI_range=np.nan)
luad["BMI_range"] = np.where(luad["BMI"] < 18.5, "underweight", luad["BMI_range"])
luad["BMI_range"] = np.where((luad["BMI"] < 25) & (luad["BMI"] >= 18.5), "healthyweight", luad["BMI_range"])
luad["BMI_range"] = np.where(luad["BMI"] >= 25, "overweight or obese", luad["BMI_range"])

In [73]:
luad["BMI_range"].value_counts(dropna=False).sort_index()

healthyweight          52
overweight or obese    40
underweight            17
Name: BMI_range, dtype: int64

### Simplify the Stage column

We had to combine the Stage 2 and 3 categories to get high enough counts to meet the assumptions of the chi squared test.

In [74]:
luad["Stage"].value_counts(dropna=False).sort_index()

1       1
1A     15
1B     29
2A      8
2B      9
3       1
3A     12
NaN    34
Name: Stage, dtype: int64

In [75]:
def simplify_stage_luad(row):
    if pd.isna(row):
        return row
    elif row.startswith("3"):
        return "2 or 3"
    elif row.startswith("2"):
        return "2 or 3"
    elif row.startswith("1"):
        return "1"
    else:
        return row
    
luad = luad.assign(Stage=luad["Stage"].apply(simplify_stage_luad))

In [76]:
luad["Stage"].value_counts(dropna=False).sort_index()

1         45
2 or 3    30
NaN       34
Name: Stage, dtype: int64

### Pack.Years.Smoked column

In [77]:
luad["Pack.Years.Smoked"] = luad["Pack.Years.Smoked"].astype(float)

luad = luad.assign(**{"Pack.Years.Smoked": (luad["Pack.Years.Smoked"] // 15) * 15})

In [78]:
luad = luad.assign(**{
    "Pack.Years.Smoked": luad["Pack.Years.Smoked"].where(
        cond=(luad["Pack.Years.Smoked"] < 15) | (pd.isnull(luad["Pack.Years.Smoked"])), other=15
    )
})

In [79]:
luad["Pack.Years.Smoked"].value_counts(dropna=False).sort_index()

0.0     21
15.0    28
NaN     60
Name: Pack.Years.Smoked, dtype: int64

### Run chi squared tests

In [80]:
luad_cols = [
    "Age_group",
    "Gender",
    "BMI_range",
    "Stage",
    "Region.of.Origin",
    "Pack.Years.Smoked"
]

In [81]:
all_results = all_results.append(test_cnv_association(
    cancer_type="luad",
    df=luad,
    test_cols=luad_cols,
    cnv_col="8p_loss"
))

In [82]:
all_results = all_results.append(test_cnv_association(
    cancer_type="luad",
    df=luad,
    test_cols=luad_cols,
    cnv_col="8q_gain"
))

## Ovarian

In [83]:
ovarian = load_tables("ovarian")

### Group ages

In [84]:
ovarian = ovarian.assign(Age=ovarian["Participant_Procurement_Age"] // 12)
ovarian = ovarian.assign(Age_group=(ovarian["Age"] // 10) * 10)

In [85]:
ovarian["Age_group"].value_counts(dropna=False).sort_index()

30.0     3
40.0    14
50.0    38
60.0    29
70.0    15
80.0     1
Name: Age_group, dtype: int64

In [86]:
ovarian = ovarian.assign(
    Age_group=ovarian["Age_group"].where(cond=(ovarian["Age_group"] < 70) | (pd.isnull(ovarian["Age"])), other=70)
)
ovarian = ovarian.assign(
    Age_group=ovarian["Age_group"].where(cond=(ovarian["Age_group"] > 40) | (pd.isnull(ovarian["Age"])), other=40)
)

In [87]:
ovarian["Age_group"].value_counts(dropna=False).sort_index()

40.0    17
50.0    38
60.0    29
70.0    16
Name: Age_group, dtype: int64

### Run chi squared tests

In [88]:
ovarian_cols = [
    "Age_group",
#     "Participant_Race", # Not enough patients in the different categories to pass the assumptions of the test.
#     "Histological_Subtype", # All are "Serous Adenocarcinoma"
#     "Tumor_Stage_Ovary_FIGO", # Not enough of different stages to pass assumptions of test
#     "Tumor_Grade" # Not enough of different grades to pass assumptions of test
]

In [89]:
all_results = all_results.append(test_cnv_association(
    cancer_type="ovarian",
    df=ovarian,
    test_cols=ovarian_cols,
    cnv_col="8p_loss"
))

In [90]:
all_results = all_results.append(test_cnv_association(
    cancer_type="ovarian",
    df=ovarian,
    test_cols=ovarian_cols,
    cnv_col="8q_gain"
))

## Multiple testing correction

In [91]:
reject, pvals_corrected, alphacSidak, alphacBonf = statsmodels.stats.multitest.multipletests(
    pvals=all_results["pval"], 
    alpha=0.05, 
    method="fdr_bh"
)

all_results = all_results.assign(adj_p=pvals_corrected)

In [92]:
all_results[all_results["adj_p"] <= 0.05]

,cancer_type,cnv_event,variable,pval,adj_p
3,brca,8q_gain,PAM50,0.001121,0.020171
4,brca,8q_gain,NMF.v2.1,0.000461,0.020171
2,luad,8q_gain,BMI_range,0.003183,0.042971
4,luad,8q_gain,Region.of.Origin,0.000801,0.020171


In [93]:
all_results

,cancer_type,cnv_event,variable,pval,adj_p
0,brca,8p_loss,Age_group,0.245893,0.613231
1,brca,8p_loss,Race,0.664741,0.892749
2,brca,8p_loss,Stage,0.973413,0.973413
3,brca,8p_loss,PAM50,0.668204,0.892749
4,brca,8p_loss,NMF.v2.1,0.937665,0.955357
0,brca,8q_gain,Age_group,0.837975,0.923482
1,brca,8q_gain,Race,0.223218,0.613231
2,brca,8q_gain,Stage,0.674630,0.892749
3,brca,8q_gain,PAM50,0.001121,0.020171
4,brca,8q_gain,NMF.v2.1,0.000461,0.020171


In [95]:
alt.Chart(all_results).mark_bar().encode(
    x=alt.X(
        "adj_p",
        bin=alt.Bin(step=0.05)
    ),
    y=alt.Y(
        "count()"
    )
)

alt.Chart(...)